In [16]:
import pandas as pd
import numpy as np
import gzip
import csv
import pickle
from multiprocessing import Pool

In [17]:
def norm_count(T_count, CDS_count, CDSlen):
    avg_CDS_count = CDS_count / CDSlen
    
    return {key: value/avg_CDS_count for key, value in T_count.items()}


def calc_avg_density_range(direc, SP, site_set, P_gtf, codon_pos_info, scope):
    print(f'{SP} {site_set} start -----')
    
    valid_ID = set(P_gtf.keys())
    P_gtf[''] = [0,0]
    
    T_norm_counts = {}
    T_ID = ''
    T_count = {}
    CDS_count = -1
    with gzip.open(f'{direc}/{SP}.rep.tagcoords.gz', 'rt') as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            if line[1] not in valid_ID : continue
            # if information of a transcript ends
            if T_ID != line[1] :
                # save normalized counts of the transcript, if it passes criterion; at least 1 CDS count per codon
                if CDS_count >= P_gtf[T_ID][1]/3 : T_norm_counts[T_ID] = norm_count(T_count, CDS_count, P_gtf[T_ID][1])
                # then reset
                T_count = {}
                CDS_count = 0
            T_ID = line[1]
            
            # counting
            if line[4]=='CDS' : CDS_count += 1
            if T_count.get(int(line[2]))==None : T_count[int(line[2])] = 0
            T_count[int(line[2])] += 1
    
    sum_band = [0.]*(scope*2+3)
    num_band = 0
    for T_ID in T_norm_counts:
        T_norm_count = T_norm_counts[T_ID]
        if codon_pos_info.get(T_ID)==None : continue
        for pos in codon_pos_info[T_ID]:
            band = [0.]*(scope*2+3)
            focus = pos-scope
            for i in range(len(band)):
                if T_norm_count.get(focus)!=None : band[i] = T_norm_count[focus]
                focus += 1
            
            sum_band = [sum(x) for x in zip(sum_band, band)]
            num_band += 1
    
    avg_band = [x/num_band for x in sum_band]
    
    print(f'{SP} {site_set} done  +++++')
    
    return [SP, site_set, avg_band]

In [18]:
if __name__ == '__main__':
    
    threads = 36
    
    direc = '/Data_2/Jun/Adipocytes/rpf/novaseq'
    SPs = [day+rep for day in ['D0','D4','D8'] for rep in ['a','b','c']]
    scope = 150
    

    # Loading stalling site info.
    with open('/Data_2/Daehwa/Adipocyte/Analysis/Ribosome_stalling/v20230730/adi_stall-score.df.pickle',"rb") as fr: stalling_score = pickle.load(fr)
    stalling_score['AA_codon'] = stalling_score['aa-asite']+' '+stalling_score['codon-asite']
    stalling_score = stalling_score.replace(0, np.nan).dropna()

    stalling_codon_pos_infos = {}
    for SP in stalling_score.iloc[:,5:-1]:
        # Top 5%
        tmp = stalling_score.sort_values(SP, ascending=False)[:(len(stalling_score)//100)*5]
        
        stalling_codon_pos_infos[f'{SP}_Top5%'] = {}
        for index, row in tmp[['transcript_id','asite']].iterrows():
            if stalling_codon_pos_infos[f'{SP}_Top5%'].get(row[0])==None : stalling_codon_pos_infos[f'{SP}_Top5%'][row[0]] = []
            stalling_codon_pos_infos[f'{SP}_Top5%'][row[0]].append(row[1])

        # Mid 5%
        tmp = stalling_score.sort_values(SP, ascending=False)[(len(stalling_score)//1000)*475:(len(stalling_score)//1000)*525]
        
        stalling_codon_pos_infos[f'{SP}_Mid5%'] = {}
        for index, row in tmp[['transcript_id','asite']].iterrows():
            if stalling_codon_pos_infos[f'{SP}_Mid5%'].get(row[0])==None : stalling_codon_pos_infos[f'{SP}_Mid5%'][row[0]] = []
            stalling_codon_pos_infos[f'{SP}_Mid5%'][row[0]].append(row[1])

        # Near1
        tmp = stalling_score.sort_values(SP, ascending=False)[(len(stalling_score)//100)*60:(len(stalling_score)//100)*65]
        
        stalling_codon_pos_infos[f'{SP}_Near1'] = {}
        for index, row in tmp[['transcript_id','asite']].iterrows():
            if stalling_codon_pos_infos[f'{SP}_Near1'].get(row[0])==None : stalling_codon_pos_infos[f'{SP}_Near1'][row[0]] = []
            stalling_codon_pos_infos[f'{SP}_Near1'][row[0]].append(row[1])

        # Bottom 5%
        tmp = stalling_score.sort_values(SP, ascending=False)[-(len(stalling_score)//100)*5:]
        
        stalling_codon_pos_infos[f'{SP}_Btm5%'] = {}
        for index, row in tmp[['transcript_id','asite']].iterrows():
            if stalling_codon_pos_infos[f'{SP}_Btm5%'].get(row[0])==None : stalling_codon_pos_infos[f'{SP}_Btm5%'][row[0]] = []
            stalling_codon_pos_infos[f'{SP}_Btm5%'][row[0]].append(row[1])
        
    # display(stalling_codon_pos_infos)


    # Loading GTF info.
    P_gtf = pd.read_csv('/Data_2/Daehwa/Data_Library/GTF_parsed/v0.7.1/gencode.vM27.annotation.gtf/Processed_gtf.tsv',
                        sep='\t', usecols=['Transcript_ID','Transcript_length','CDS_length'])
    P_gtf = P_gtf[P_gtf["CDS_length"]>0]
    P_gtf = P_gtf[P_gtf["CDS_length"]%3==0]
    P_gtf = P_gtf.set_index('Transcript_ID').T.to_dict('list')
    
    # Run the analysis
    args = [(direc, site_set[:3], site_set, P_gtf, stalling_codon_pos_infos[site_set], scope) for site_set in stalling_codon_pos_infos]
    p = Pool(processes=threads)
    results = p.starmap(calc_avg_density_range, args)
    p.close()
    
    # organization of results
    result_org = {TB:{SP:None for SP in SPs} for TB in ['Top5%','Mid5%','Near1','Btm5%']}
    for result in results:
        result_org[result[1][-5:]][result[0]] = result[2]
    
    # Save as 
    out_w = pd.ExcelWriter(f'adi_top-stalling-score_metagene.xlsx')
    for TB in result_org:
        pd.DataFrame(result_org[TB]).to_excel(out_w, sheet_name=TB, index=False)
    out_w.close()

D0a D0a_Top5% start -----
D0a D0a_Mid5% start -----
D0a D0a_Btm5% start -----
D0b D0b_Top5% start -----
D0b D0b_Mid5% start -----
D0b D0b_Btm5% start -----
D0c D0c_Top5% start -----
D0c D0c_Mid5% start -----
D0c D0c_Btm5% start -----
D4a D4a_Top5% start -----
D4a D4a_Mid5% start -----
D4a D4a_Btm5% start -----
D4b D4b_Top5% start -----
D4b D4b_Mid5% start -----
D4b D4b_Btm5% start -----
D4c D4c_Top5% start -----
D4c D4c_Mid5% start -----
D4c D4c_Btm5% start -----
D8a D8a_Top5% start -----
D8a D8a_Mid5% start -----
D8a D8a_Btm5% start -----
D8b D8b_Top5% start -----
D8b D8b_Mid5% start -----
D8b D8b_Btm5% start -----
D8c D8c_Top5% start -----
D8c D8c_Mid5% start -----
D8c D8c_Btm5% start -----
D0b D0b_Mid5% done  +++++
D0a D0a_Top5% done  +++++
D0a D0a_Mid5% done  +++++
D0b D0b_Top5% done  +++++
D0b D0b_Btm5% done  +++++
D0a D0a_Btm5% done  +++++
D0c D0c_Top5% done  +++++
D0c D0c_Mid5% done  +++++
D8a D8a_Mid5% done  +++++
D8a D8a_Top5% done  +++++
D0c D0c_Btm5% done  +++++
D8a D8a_Btm5